# Setup Environment

## Install KVM on Kernel 5.4.24-1-MANJARO x86_64
https://www.fosslinux.com/2484/how-to-install-virtual-machine-manager-kvm-in-manjaro-and-arch-linux.htm   
https://computingforgeeks.com/complete-installation-of-kvmqemu-and-virt-manager-on-arch-linux-and-manjaro/

## Download Ubuntu ISO Bionic
http://releases.ubuntu.com/18.04.4/ubuntu-18.04.4-desktop-amd64.iso

## QEMU
Documentation: https://wiki.archlinux.org/index.php/QEMU   
Networking arch: https://wiki.qemu.org/Documentation/Networking

### Create a Disk Image
```
sudo qemu-img create -f qcow2 /var/lib/libvirt/images/pool/openair-cn 10G
```

### Launch QEMU with Ubuntu ISO attached
Run the following command to launch a QEMU virtual machine with the Ubuntu ISO and hard disk we created in the previous step attached.

```
sudo qemu-system-x86_64 \
  -m 4096 \
  -smp 2 \
  -vga virtio \
  -show-cursor \
  -usb \
  -device usb-tablet \
  -enable-kvm \
  -cdrom ubuntu-18.04.4-desktop-amd64.iso \
  -drive file=/var/lib/libvirt/images/pool/openair-cn,format=qcow2,if=virtio \
  -cpu host

```
**The iso must be located in the same place where the command is executed.**

Use _-monitor stdio_ to open monitor.

### Reboot without Ubuntu ISO attached
Once the machine is powered off. Run the command below to power it back on without the CD-ROM attached.

```
sudo qemu-system-x86_64 \
  -m 4096 \
  -smp 2 \
  -vga virtio \
  -show-cursor \
  -usb \
  -device usb-tablet \
  -enable-kvm \
  -drive file=/var/lib/libvirt/images/pool/openair-cn,format=qcow2,if=virtio \
  -cpu host \
  -net nic,model=virtio \
  -net user,hostfwd=tcp::2222-:22 \
  -netdev user,id=enp0s20f0u4c4i2,net=192.168.76.0/24,dhcpstart=192.168.76.9
```

Ref: https://graspingtech.com/ubuntu-desktop-18.04-virtual-machine-macos-qemu/

TODO: Start ubuntu in console mode
https://askubuntu.com/questions/859630/how-to-start-ubuntu-in-console-mode

## Install Packages in Guest 
In guest, open terminal.

```
sudo apt update
sudo apt upgrade -y
sudo apt install openssh-server git curl vim tmux
```

## Access Guest from Host
Open terminal and you copy public key and then loggin on guest.

```
ssh-copy-id openair-cn@localhost -p 2222
ssh openair-cn@localhost -p 2222
```

# Open Air Interface Core Network ( OAI-CN )
Cool tool:
https://www.gitmemory.com/openairinterface-GH

Tutorial Cassandra: 
https://www.tutorialspoint.com/cassandra/cassandra_architecture.htm

**The next steps were based on the following links:**
* https://github.com/OPENAIRINTERFACE/openair-cn/wiki/OpenAirSoftwareSupport
* https://github.com/OPENAIRINTERFACE/openair-cn-cups/wiki/OpenAirSoftwareSupport

## Network overview

![Network Overview](https://github.com/OPENAIRINTERFACE/openair-cn-cups/wiki/pictures/oai_epc_all_in_one_cups_spgw.png)

## Add key
You must trust the key below in order to have success.

```
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A278B781FE4B2BDA
```
## Enable ssh
Generate ssh keys
```
ssh-keygen -t dsa
```
Enable password authentication

In /etc/ssh/sshd_config set
```
PasswordAuthentication yes 
```

Then you can also copy your ssh public key in the VM...

## Download EPC source code


```
ubuntu@test-install-openair-cn:~$ git clone https://github.com/OPENAIRINTERFACE/openair-cn.git
...
ubuntu@test-install-openair-cn:~$ git clone https://github.com/OPENAIRINTERFACE/openair-cn-cups.git
...
```

## Checkout Realeases
This tutorial was tested with **release 2020.w13** in openair-cn.git   
This tutorial was tested with **release 2019.w47** in openair-cn-cups.git

## Install HSS  
**BGN - This part of the wiki is not working due to the java's ppa. Oracle stopped to support it.**

```
ubuntu@test-install-openair-cn:~$ cd openair-cn/scripts/
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ ./build_cassandra --check-installed-software --force
...
Processing triggers for libgdk-pixbuf2.0-0:amd64 (2.36.11-2) ...
Cassandra installation successful
'/home/ubuntu/openair-cn/etc/cassandra.conf' -> '/usr/local/etc/oai/cassandra.conf'
Cassandra has been installed successfully.
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ 
```

~~Set the JRE version 8 as default~~
```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra stop
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo update-alternatives --config java
# select jre version 8
... 
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra start
```
Verify that Cassandra is installed and running

**END - This part of the wiki is not working due to the java's ppa. Oracle stopped to support it.**

The message below is shown during the execution of https://github.com/OPENAIRINTERFACE/openair-cn/wiki/OpenAirSoftwareSupport#install-hss, it seems to be missing oracle-java8-installer and there is no valid OpenPGP data.

```
Package oracle-java8-installer is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'oracle-java8-installer' has no installation candidate
deb http://www.apache.org/dist/cassandra/debian 21x main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   227  100   227    0     0    378      0 --:--:-- --:--:-- --:--:--   377
gpg: no valid OpenPGP data found.
```


**See Troubleshoot to fix it**

**Now, we can repeat the steps again**

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra stop
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo update-alternatives --config java
# select jre version 8
... 
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra start
```
Verify that Cassandra is installed and running

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ nodetool status
```

The output should look something like:

```
Datacenter: datacenter1
=======================
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack
UN  127.0.0.1  51.69 KB   256     100.0%            fa761e10-4118-4a99-9a2e-829e7cdd8e82  rack1
```

Stop Cassandra and cleanup the log files before modifying the configuration

```
sudo service cassandra stop
sudo rm -rf /var/lib/cassandra/data/system/*
sudo rm -rf /var/lib/cassandra/commitlog/*
sudo rm -rf /var/lib/cassandra/data/system_traces/*
sudo rm -rf /var/lib/cassandra/saved_caches/*
```

Update the Cassandra configuration **if needed** (default configuration set **1 cassandra node listening on localhost**)
Update **/etc/cassandra/cassandra.yaml** as indicated below. The <Cassandra_Server_IP> address should be the IP
address of the Cassandra server that the HSS will use to connect to Cassandra. The "..." below indicate configuration lines between values that need to be modified.
```
...
cluster_name: "HSS Cluster"
...
# OAI Note: no to change Cassandra_Server_IP that is already 127.0.0.1
seed_provider:
- class_name: org.apache.cassandra.locator.SimpleSeedProvider
- seeds: "<Cassandra_Server_IP>"
...
listen_address: <Cassandra_Server_IP>
...
rpc_address: <Cassandra_Server_IP>
...
# OAI Note: seems to be this option that is working ()
# endpoint_snitch: SimpleSnitch
endpoint_snitch: GossipingPropertyFileSnitch
```

**Start Cassandra service**

```
sudo service cassandra start
```

**Check cassandra service status**

```
● cassandra.service - LSB: distributed storage system for structured data
   Loaded: loaded (/etc/init.d/cassandra; generated)
   Active: active (running) since Thu 2019-05-23 10:26:51 UTC; 1h 54min ago
     Docs: man:systemd-sysv-generator(8)
    Tasks: 61 (limit: 4704)
   CGroup: /system.slice/cassandra.service
           └─23863 java -ea -javaagent:/usr/share/cassandra/lib/jamm-0.3.0.jar -XX:+CMSClassUnloadingEnabled -XX:+UseThreadPri

May 23 10:26:51 test-install-openair-cn systemd[1]: Starting LSB: distributed storage system for structured data...
May 23 10:26:51 test-install-openair-cn systemd[1]: Started LSB: distributed storage system for structured data.
```

**Install HSS software dependancies**

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ ./build_hss_rel14 --check-installed-software --force
...
# OAI Note: Takes some time...
```
Output is:
```
...
-- Installing: /usr/local/lib/cmake/pistache/PistacheConfig.cmake
~/openair-cn/scripts
HSS software installation successful
HSS not compiled, to compile it, re-run build_hss without -i option
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ 
```

**Build HSS**

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ ./build_hss_rel14 --clean
```

Output is:

```
mkdir: created directory '/home/ubuntu/openair-cn/scripts/../build/hss_rel14/build'
~/openair-cn/build/hss_rel14/util ~/openair-cn/scripts
...
 g++  -g -pthread -std=c++11  -I ../../src/hss_rel14/include -I ../../src/hss_rel14/util/include -I ../../build/hss_rel14/../git_submodules/rapidjson/include -I /usr/local/include/freeDiameter -I ../../src/hss_rel14/hsssec/include -I /usr/local/include -MMD -c -o ../../build/hss_rel14/build/s6t_impl.o ../../src/hss_rel14/src/s6t_impl.cpp
 Linking...
 g++  -g -pthread -lpthread -Wl,-rpath,/usr/local/lib/x86_64-linux-gnu:/usr/local/lib ../../build/hss_rel14/build/s6t.o ../../build/hss_rel14/build/s6as6d_impl.o ../../build/hss_rel14/build/dataaccess.o ../../build/hss_rel14/build/s6as6d.o ../../build/hss_rel14/build/options.o ../../build/hss_rel14/build/s6c_impl.o ../../build/hss_rel14/build/fdhss.o ../../build/hss_rel14/build/s6c.o ../../build/hss_rel14/build/main.o ../../build/hss_rel14/build/util.o ../../build/hss_rel14/build/resthandler.o ../../build/hss_rel14/build/s6t_impl.o -o ../../build/hss_rel14/bin/hss ../../build/hss_rel14/util/lib/libc3po.a ../../build/hss_rel14/hsssec/lib/libhsssec.a /usr/local/lib/libpistache.a /usr/local/lib/libfdcore.so /usr/local/lib/libfdproto.so -L/usr/local/lib/x86_64-linux-gnu -lcassandra -lrt -lnettle -lgmp 
'/home/ubuntu/openair-cn/build/hss_rel14/bin/hss' -> '/usr/local/bin/oai_hss'
oai_hss installed

```

###  Populate users table
Here you will have to feed scripts with your parameters:

!!! IMPORTANT NOTICE !!! It is highly recommended that your APN contains a realm, else you will have to enter in the SPGW config file ...gprs
APN (Access Point Name): --apn value # value is "default.ng4T.com" here.

LTE K:                   --key value # value is "fec86ba6eb707ed08905757b1bb44b8f" here.

IMSI:                    --imsi value # value of IMSI of first subscriber is "208931234561000" here, the IMSI of next subscribers will be incremented by one, if --no-of-users value is > 1.

REALM:                   --realm value # value of realm here is ng4T.com.

MME FQDN:                --mme-identity value # value of MME FQDN is mme.ng4T.com here (end with realm).


```
Cassandra_Server_IP='127.0.0.1'
cqlsh --file ../src/hss_rel14/db/oai_db.cql $Cassandra_Server_IP
./data_provisioning_users --apn default.ng4T.com --apn2 internet --key fec86ba6eb707ed08905757b1bb44b8f --imsi-first 208931234561000 --msisdn-first 001011234561000 --mme-identity mme.ng4T.com --no-of-users 20 --realm ng4T.com --truncate True  --verbose True --cassandra-cluster $Cassandra_Server_IP
```

Output is 
```
INFO:root:208931234561005 1011234561005 41 fec86ba6eb707ed08905757b1bb44b8f mme.ng4T.com 3 ng4T.com 2683b376d1056746de3b254012908e0e 96 {"Subscription-Data":{"Access-Restriction-Data":41,"Subscriber-Status":0,"Network-Access-Mode":2,"Regional-Subscription-Zone-Code":["0x0123","0x4567","0x89AB","0xCDEF","0x1234","0x5678","0x9ABC","0xDEF0","0x2345","0x6789"],"MSISDN":"0x1011234561005","AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"APN-Configuration-Profile":{"Context-Identifier":0,"All-APN-Configurations-Included-Indicator":0,"APN-Configuration":{"Context-Identifier":0,"PDN-Type":0,"Service-Selection":"default.ng4T.com","EPS-Subscribed-QoS-Profile":{"QoS-Class-Identifier":9,"Allocation-Retention-Priority":{"Priority-Level":15,"Pre-emption-Capability":0,"Pre-emption-Vulnerability":0}},"AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"PDN-GW-Allocation-Type":0,"MIP6-Agent-Info":{"MIP-Home-Agent-Address":["172.26.17.183"]}},"APN-Configuration":{"Context-Identifier":0,"PDN-Type":0,"Service-Selection":"internet","EPS-Subscribed-QoS-Profile":{"QoS-Class-Identifier":9,"Allocation-Retention-Priority":{"Priority-Level":13,"Pre-emption-Capability":1,"Pre-emption-Vulnerability":0}},"AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"PDN-GW-Allocation-Type":0,"MIP6-Agent-Info":{"MIP-Home-Agent-Address":["172.26.17.183"]}}},"Subscribed-Periodic-RAU-TAU-Timer":0}}

...
INFO:root:1011234561007 208931234561007
INFO:root:1011234561002 208931234561002
INFO:root:1011234561009 208931234561009
INFO:root:1011234561005 208931234561005
INFO:root:1011234561014 208931234561014
INFO:root:1011234561011 208931234561011
INFO:root:1011234561003 208931234561003
INFO:root:1011234561016 208931234561016
INFO:root:1011234561015 208931234561015
INFO:root:1011234561010 208931234561010
INFO:root:1011234561008 208931234561008
INFO:root:1011234561013 208931234561013
INFO:root:1011234561018 208931234561018
INFO:root:1011234561017 208931234561017
INFO:root:1011234561006 208931234561006
INFO:root:1011234561019 208931234561019
INFO:root:1011234561012 208931234561012
INFO:root:1011234561001 208931234561001
INFO:root:1011234561000 208931234561000
INFO:root:1011234561004 208931234561004
DEBUG:cassandra.io.libevreactor:Closing connection (140287699882392) to 127.0.0.1
DEBUG:cassandra.io.libevreactor:Closed socket to 127.0.0.1
DEBUG:cassandra.io.libevreactor:Closing connection (140287699883512) to 127.0.0.1
DEBUG:cassandra.io.libevreactor:Closed socket to 127.0.0.1
DEBUG:cassandra.io.libevreactor:Waiting for event loop thread to join...
DEBUG:cassandra.io.libevreactor:All Connections currently closed, event loop ended
DEBUG:cassandra.io.libevreactor:Event loop thread was joined
DEBUG:cassandra.cluster:Shutting down Cluster Scheduler
DEBUG:cassandra.cluster:Shutting down control connection
```

```
./data_provisioning_mme --id 3 --mme-identity mme.ng4T.com --realm ng4T.com --ue-reachability 1 --truncate True  --verbose True -C $Cassandra_Server_IP
```

Output is 

```
vhss.mmeidentity truncated
vhss.mmeidentity_host truncated
3 mme-isdn mme.ng4T.com ng4T.com 1
3 mme-isdn mme.ng4T.com ng4T.com 1
```

**Create HSS configuration files**

```
# prompt has been removed for easier Ctrl+C Ctrl+V
# cd $OPENAIRCN_DIR/scripts

openssl rand -out $HOME/.rnd 128

PREFIX='/usr/local/etc/oai'
sudo mkdir -p $PREFIX
sudo chmod 777 $PREFIX
sudo mkdir  $PREFIX/freeDiameter
sudo chmod 777 $PREFIX/freeDiameter

# freeDiameter configuration files
sudo cp ../etc/acl.conf ../etc/hss_rel14_fd.conf $PREFIX/freeDiameter
sudo chmod 666 $PREFIX/freeDiameter/acl.conf $PREFIX/freeDiameter/hss_rel14_fd.conf

# HSS configuration files
sudo cp ../etc/hss_rel14.conf ../etc/hss_rel14.json $PREFIX
sudo chmod 666 $PREFIX/hss_rel14.conf $PREFIX/hss_rel14.json

declare -A HSS_CONF
HSS_CONF[@PREFIX@]=$PREFIX
HSS_CONF[@REALM@]='ng4T.com'
HSS_CONF[@HSS_FQDN@]="hss.${HSS_CONF[@REALM@]}"
HSS_CONF[@cassandra_Server_IP@]='127.0.0.1' 
HSS_CONF[@cassandra_IP@]='127.0.0.1' # TODO harmonize these 2
HSS_CONF[@OP_KEY@]='1006020f0a478bf6b699f15c062e42b3'
HSS_CONF[@ROAMING_ALLOWED@]='true'

for K in "${!HSS_CONF[@]}"; do 
  egrep -lRZ "$K" $PREFIX | xargs -0 -l sed -i -e "s|$K|${HSS_CONF[$K]}|g"
done

### freeDiameter certificate
../src/hss_rel14/bin/make_certs.sh hss ${HSS_CONF[@REALM@]} $PREFIX

# Finally customize the listen address of FD server (if necessary because FD binds by default on available interfaces)
# set in $PREFIX/freeDiameter/hss_rel14_fd.conf and uncomment the following line
sudo sed -i -e 's/#ListenOn/ListenOn/g' $PREFIX/freeDiameter/hss_rel14_fd.conf 
```
You can ignore the erros for while on the execution of _make_certs.sh_

```
Can't open ./demoCA/index.txt.attr for reading, No such file or directory
140137160171968:error:02001002:system library:fopen:No such file or directory:../crypto/bio/bss_file.c:72:fopen('./demoCA/index.txt.attr','r')
140137160171968:error:2006D080:BIO routines:BIO_new_file:no such file:../crypto/bio/bss_file.c:79:
```

### Update OPC (Operator Key Ciphered) Cassandra DB if necessary (do it almost one time unless you have only OPC and populate it in DB)
```
oai_hss -j $PREFIX/hss_rel14.json --onlyloadkey
```

Output is 
```
Options::jsonConfig            : /usr/local/etc/oai/hss_rel14.json
Options::diameterconfiguration : /usr/local/etc/oai/freeDiameter/hss_rel14_fd.conf
Options::cassserver            : 127.0.0.1
Options::cassuser              : root
Options::casspwd               : root
Options::cassdb                : vhss
Options::randvector            : 1
Options::roamallow             : 1
Options::optkey                : 1006020f0a478bf6b699f15c062e42b3
Options::reloadkey             : 0
Options::onlyloadkey           : 1
Options::gtwport               : 9080
Options::gtwhost               : *
Options::resthost              : 9081
Options::synchimsi             : 
Options::synchauts             : 
Initialized random


******************************************
IMSI:208931234561005
KEY :fec86ba6eb707ed08905757b1bb44b8f
OPC :
RijndaelKeySchedule: K FEC86BA6EB707ED08905757B1BB44B8F
Compute opc:
	K:	FEC86BA6EB707ED08905757B1BB44B8F
	In:	1006020F0A478BF6B699F15C062E42B3
	Rinj:	D4224B3931FD5BDDD0489A9573F93E72
	Out:	C42449363BBAD02B66D16BC975D77CC1
NEW OPC :c42449363bbad02b66d16bc975d77cc1
******************************************
UPDATE vhss.users_imsi SET OPc='c42449363bbad02b66d16bc975d77cc1' WHERE imsi='208931234561005';
******************************************
...
NEW OPC :c42449363bbad02b66d16bc975d77cc1
******************************************
UPDATE vhss.users_imsi SET OPc='c42449363bbad02b66d16bc975d77cc1' WHERE imsi='208931234561007';
```

Note that if option reloadkey is set to true in $PREFIX/hss_rel14.json, this step is not required.

If you face the error  below, go to Trobleshoot to fix it 
```
openair-cn@openair-cn-vm:~/openair-cn/scripts$ oai_hss -j $PREFIX/hss_rel14.json --onlyloadkey
terminate called after throwing an instance of 'spdlog::spdlog_ex'
  what():  Failed opening file logs/hss.log for writing: No such file or directory
Aborted (core dumped)

```

### Kill HSS
```
sudo kill -9 $(ps -aux | grep hss | awk '{print $2}')
```

## Install MME

**Install MME software dependencies**

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ ./build_mme --check-installed-software --force
```

Output is
```
...
Processing triggers for mime-support (3.60ubuntu1) ...
Setting up unzip (6.0-21ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
sh: 0: getcwd() failed: No such file or directory
MME software installation successful
MME not compiled, to compile it, re-run build_mme without -i option
```

**Build MME**

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ ./build_mme --clean
```
Ouput is
```
Clean the build generated files (build from scratch)
mkdir: created directory 'build'
Build type is Debug
Architecture is x86_64
git found: /usr/bin/git
NETTLE VERSION_INSTALLED  = 3.4
NETTLE_VERSION_MAJOR = 3
NETTLE_VERSION_MINOR = 4
Compilation log for /home/ubuntu/openair-cn/build/mme/build/mme is here: /home/ubuntu/openair-cn/build/log/mme.txt
mme compiled
'/home/ubuntu/openair-cn/build/mme/build/mme' -> '/usr/local/bin/mme'
mme installed
```

**Create MME configuration files**

```
# prompt has been removed for easier Ctrl+C Ctrl+V
openssl rand -out $HOME/.rnd 128
# cd $OPENAIRCN_DIR/scripts

# S6a
sudo ip addr add 172.16.1.102/24 dev ens3 label ens3:m11
sudo ip addr add 192.168.247.102/24 dev ens3 label ens3:m1c

INSTANCE=1
PREFIX='/usr/local/etc/oai'
sudo mkdir -m 0777 -p $PREFIX
sudo mkdir -m 0777    $PREFIX/freeDiameter

# freeDiameter configuration file
cp ../etc/mme_fd.sprint.conf  $PREFIX/freeDiameter/mme_fd.conf

cp ../etc/mme.conf  $PREFIX

declare -A MME_CONF

MME_CONF[@MME_S6A_IP_ADDR@]="127.0.0.11"
MME_CONF[@INSTANCE@]=$INSTANCE
MME_CONF[@PREFIX@]=$PREFIX
MME_CONF[@REALM@]='ng4T.com'
MME_CONF[@PID_DIRECTORY@]='/var/run'
MME_CONF[@MME_FQDN@]="mme.${MME_CONF[@REALM@]}"
MME_CONF[@HSS_HOSTNAME@]='hss'
MME_CONF[@HSS_FQDN@]="${MME_CONF[@HSS_HOSTNAME@]}.${MME_CONF[@REALM@]}"
MME_CONF[@HSS_IP_ADDR@]='127.0.0.1'
MME_CONF[@MCC@]='208'
MME_CONF[@MNC@]='93'
MME_CONF[@MME_GID@]='32768'
MME_CONF[@MME_CODE@]='3'
MME_CONF[@TAC_0@]='600'
MME_CONF[@TAC_1@]='601'
MME_CONF[@TAC_2@]='602'
MME_CONF[@MME_INTERFACE_NAME_FOR_S1_MME@]='ens3:m1c'
MME_CONF[@MME_IPV4_ADDRESS_FOR_S1_MME@]='192.168.247.102/24'
MME_CONF[@MME_INTERFACE_NAME_FOR_S11@]='ens3:m11'
MME_CONF[@MME_IPV4_ADDRESS_FOR_S11@]='172.16.1.102/24'
MME_CONF[@MME_INTERFACE_NAME_FOR_S10@]='ens3:m10'
MME_CONF[@MME_IPV4_ADDRESS_FOR_S10@]='192.168.10.110/24'
MME_CONF[@OUTPUT@]='CONSOLE'
MME_CONF[@SGW_IPV4_ADDRESS_FOR_S11_TEST_0@]='172.16.1.104/24'
MME_CONF[@SGW_IPV4_ADDRESS_FOR_S11_0@]='172.16.1.104/24'
MME_CONF[@PEER_MME_IPV4_ADDRESS_FOR_S10_0@]='0.0.0.0/24'
MME_CONF[@PEER_MME_IPV4_ADDRESS_FOR_S10_1@]='0.0.0.0/24'

#implicit MCC MNC 001 01
TAC_SGW_TEST='7'
tmph=`echo "$TAC_SGW_TEST / 256" | bc`
tmpl=`echo "$TAC_SGW_TEST % 256" | bc`
MME_CONF[@TAC-LB_SGW_TEST_0@]=`printf "%02x\n" $tmpl`
MME_CONF[@TAC-HB_SGW_TEST_0@]=`printf "%02x\n" $tmph`

MME_CONF[@MCC_SGW_0@]=${MME_CONF[@MCC@]}
MME_CONF[@MNC3_SGW_0@]=`printf "%03d\n" $(echo ${MME_CONF[@MNC@]} | sed 's/^0*//')`
TAC_SGW_0='600'
tmph=`echo "$TAC_SGW_0 / 256" | bc`
tmpl=`echo "$TAC_SGW_0 % 256" | bc`
MME_CONF[@TAC-LB_SGW_0@]=`printf "%02x\n" $tmpl`
MME_CONF[@TAC-HB_SGW_0@]=`printf "%02x\n" $tmph`

MME_CONF[@MCC_MME_0@]=${MME_CONF[@MCC@]}
MME_CONF[@MNC3_MME_0@]=`printf "%03d\n" $(echo ${MME_CONF[@MNC@]} | sed 's/^0*//')`
TAC_MME_0='601'
tmph=`echo "$TAC_MME_0 / 256" | bc`
tmpl=`echo "$TAC_MME_0 % 256" | bc`
MME_CONF[@TAC-LB_MME_0@]=`printf "%02x\n" $tmpl`
MME_CONF[@TAC-HB_MME_0@]=`printf "%02x\n" $tmph`

MME_CONF[@MCC_MME_1@]=${MME_CONF[@MCC@]}
MME_CONF[@MNC3_MME_1@]=`printf "%03d\n" $(echo ${MME_CONF[@MNC@]} | sed 's/^0*//')`
TAC_MME_1='602'
tmph=`echo "$TAC_MME_1 / 256" | bc`
tmpl=`echo "$TAC_MME_1 % 256" | bc`
MME_CONF[@TAC-LB_MME_1@]=`printf "%02x\n" $tmpl`
MME_CONF[@TAC-HB_MME_1@]=`printf "%02x\n" $tmph`


for K in "${!MME_CONF[@]}"; do 
  egrep -lRZ "$K" $PREFIX | xargs -0 -l sed -i -e "s|$K|${MME_CONF[$K]}|g"
  ret=$?;[[ ret -ne 0 ]] && echo "Tried to replace $K with ${MME_CONF[$K]}"
done

# Generate freeDiameter certificate
sudo ./check_mme_s6a_certificate $PREFIX/freeDiameter mme.${MME_CONF[@REALM@]}
```

## Install SPGW-U
**Install SPGW-U software dependencies**

```
ubuntu@test-install-openair-cn:~$ cd ~/openair-cn-cups/build/scripts; ./build_spgwu -I -f
```

**If the error below was shown, go to Troubleshooting**:


**Build SPGW-U**

As a starting point, you can start with a Debug build, then you can change the build type
Check build types with './build_spgwu -h'

```
ubuntu@test-install-openair-cn:~/openair-cn-cups/build/scripts$ cd ~/openair-cn-cups/build/scripts; ./build_spgwu -c -V -b Debug -j
```

Output is 

```
Clean the build generated files (build from scratch)
CMake build process verbose
Cleaning S/P-GW-U: generated configuration files, obj files, executable
mkdir: created directory 'build'
Build type is Debug
Architecture is x86_64
git found: /usr/bin/git
Compilation log for /home/ubuntu/openair-cn-cups/build/spgw_u/build/spgwu is here: /home/ubuntu/openair-cn-cups/build/log/spgwu.txt
Scanning dependencies of target 3GPP_COMMON_TYPES
[  2%] Building CXX object common/CMakeFiles/3GPP_COMMON_TYPES.dir/logger.cpp.o
...
[ 97%] Building CXX object CMakeFiles/spgwu.dir/home/ubuntu/openair-cn-cups/src/itti/itti_msg.cpp.o
[100%] Linking CXX executable spgwu
[100%] Built target spgwu
spgwu compiled
'/home/ubuntu/openair-cn-cups/build/spgw_u/build/spgwu' -> '/usr/local/bin/spgwu'
spgwu installed
```

**Create SPGW-U configuration files**

```
# prompt has been removed for easier Ctrl+C Ctrl+V
sudo ip addr add 172.55.55.102/24 dev ens3 label ens3:sxu # SPGW-U SXab interface
sudo ip addr add 192.168.248.159/24 dev ens3 label ens3:s1u # SPGW-U S1U interface
INSTANCE=1
PREFIX='/usr/local/etc/oai'
sudo mkdir -m 0777 -p $PREFIX
cp ../../etc/spgw_u.conf  $PREFIX

declare -A SPGWU_CONF

SPGWU_CONF[@INSTANCE@]=$INSTANCE
SPGWU_CONF[@PREFIX@]=$PREFIX
SPGWU_CONF[@PID_DIRECTORY@]='/var/run'
SPGWU_CONF[@SGW_INTERFACE_NAME_FOR_S1U_S12_S4_UP@]='ens3:s1u'
SPGWU_CONF[@SGW_INTERFACE_NAME_FOR_SX@]='ens3:sxu'
SPGWU_CONF[@SGW_INTERFACE_NAME_FOR_SGI@]='ens3'

for K in "${!SPGWU_CONF[@]}"; do 
  egrep -lRZ "$K" $PREFIX | xargs -0 -l sed -i -e "s|$K|${SPGWU_CONF[$K]}|g"
  ret=$?;[[ ret -ne 0 ]] && echo "Tried to replace $K with ${SPGWU_CONF[$K]}"
done
```

Important:
Customize in $PREFIX/spgw_u.conf file the "SPGW-C_LIST" and "PDN_NETWORK_LIST" sections.

## Install SPGW-C
As a starting point, you can start with a Debug build, then you can change the build type
Check build types with './build_spgwc -h'

```
ubuntu@test-install-openair-cn:~/openair-cn-cups/build/scripts$ cd ~/openair-cn-cups/build/scripts; ./build_spgwc -c -V -b Debug -j
```

Output is 

```
Clean the build generated files (build from scratch)
CMake build process verbose
Cleaning S/P-GW-C: generated configuration files, obj files, executable
mkdir: created directory 'build'
Build type is Debug
Architecture is x86_64
git found: /usr/bin/git
Compilation log for /home/ubuntu/openair-cn-cups/build/spgw_c/build/spgwc is here: /home/ubuntu/openair-cn-cups/build/log/spgwc.txt
Scanning dependencies of target 3GPP_COMMON_TYPES
[  2%] Building CXX object common/CMakeFiles/3GPP_COMMON_TYPES.dir/logger.cpp.o
Scanning dependencies of target PGWC
...
Scanning dependencies of target spgwc
[ 92%] Building CXX object CMakeFiles/spgwc.dir/home/ubuntu/openair-cn-cups/src/oai_spgwc/options.cpp.o
[ 92%] Building CXX object CMakeFiles/spgwc.dir/home/ubuntu/openair-cn-cups/src/oai_spgwc/main.cpp.o
[ 95%] Building CXX object CMakeFiles/spgwc.dir/home/ubuntu/openair-cn-cups/src/itti/itti.cpp.o
[ 97%] Building CXX object CMakeFiles/spgwc.dir/home/ubuntu/openair-cn-cups/src/itti/itti_msg.cpp.o
[100%] Linking CXX executable spgwc
[100%] Built target spgwc
spgwc compiled
'/home/ubuntu/openair-cn-cups/build/spgw_c/build/spgwc' -> '/usr/local/bin/spgwc'
spgwc installed
```

**Create SPGW-C configuration files**

```
# prompt has been removed for easier Ctrl+C Ctrl+V
sudo ip addr add 172.55.55.101 dev ens3 label ens3:sxc # SPGW-C SXab interface
sudo ip addr add 172.58.58.102 dev ens3 label ens3:s5c # SGW-C S5S8 interface
sudo ip addr add 172.58.58.101 dev ens3 label ens3:p5c # PGW-C S5S8 interface
sudo ip addr add 172.16.1.104 dev ens3 label ens3:s11 # SGW-C S11 interface
INSTANCE=1
PREFIX='/usr/local/etc/oai'
sudo mkdir -m 0777 -p $PREFIX
cp ../../etc/spgw_c.conf  $PREFIX

declare -A SPGWC_CONF

SPGWC_CONF[@INSTANCE@]=$INSTANCE
SPGWC_CONF[@PREFIX@]=$PREFIX
SPGWC_CONF[@PID_DIRECTORY@]='/var/run'
SPGWC_CONF[@SGW_INTERFACE_NAME_FOR_S11@]='ens3:s11'
SPGWC_CONF[@SGW_INTERFACE_NAME_FOR_S5_S8_CP@]='ens3:s5c'
SPGWC_CONF[@PGW_INTERFACE_NAME_FOR_S5_S8_CP@]='ens3:p5c'
SPGWC_CONF[@PGW_INTERFACE_NAME_FOR_SX@]='ens3:sxc'
SPGWC_CONF[@DEFAULT_DNS_IPV4_ADDRESS@]='8.8.8.8'
SPGWC_CONF[@DEFAULT_DNS_SEC_IPV4_ADDRESS@]='4.4.4.4'

for K in "${!SPGWC_CONF[@]}"; do 
  egrep -lRZ "$K" $PREFIX | xargs -0 -l sed -i -e "s|$K|${SPGWC_CONF[$K]}|g"
  ret=$?;[[ ret -ne 0 ]] && echo "Tried to replace $K with ${SPGWC_CONF[$K]}"
done
```

Important:
Customize in $PREFIX/spgw_c.conf file the "IP_ADDRESS_POOL" and "APN_LIST" sections.

### Run EPC Network Functions
**Run HSS**

```
oai_hss -j $PREFIX/hss_rel14.json
```
Output is 
```
Options::jsonConfig            : /usr/local/etc/oai/hss_rel14.json
Options::diameterconfiguration : /usr/local/etc/oai/freeDiameter/hss_rel14_fd.conf
Options::cassserver            : 127.0.0.1
Options::cassuser              : root
Options::casspwd               : root
Options::cassdb                : vhss
Options::randvector            : 1
Options::roamallow             : 1
Options::optkey                : 1006020f0a478bf6b699f15c062e42b3
Options::reloadkey             : 0
Options::onlyloadkey           : 0
Options::gtwport               : 9080
Options::gtwhost               : *
Options::resthost              : 9081
Options::synchimsi             : 
Options::synchauts             : 
Initialized random
12:59:13  NOTI   libfdproto '1.2.1' initialized.
12:59:13  NOTI   libgnutls '3.5.18' initialized.
12:59:13  NOTI   HASHLIST is enabled
...
12:59:13  NOTI   Loaded extensions: '/usr/local/lib/freeDiameter/dict_S6c.fdx'[(no config file)], loaded
Connecting to cassandra host: 127.0.0.1
Registering s6t dictionary
Registering s6t command handlers
Registering COIR command handler
Registering NIIR command handler
Registering s6as6d dictionary
Registering s6as6d command handlers
Registering UPLR command handler
Registering AUIR command handler
Registering PUUR command handler
Registering s6c dictionary
Registering s6c command handlers
Registering SERIFSR command handler
12:59:13  NOTI   Local server address(es): 192.168.123.158{---L-}
The freeDiameter engine has been started
Started REST server on port [9081]
```

**Run MME**

```
./run_mme --config-file $PREFIX/mme.conf --set-virt-if
```
Output is 
```
setting config file to: /usr/local/etc/oai/mme.conf
OPENAIRCN_DIR    = /home/ubuntu/openair-cn
Configuring ens3:m1c  192.168.247.102/24
Configuring ens3:m11  172.16.1.102/24
Configuring ens3:m10  192.168.10.110/24
Initializing shared logging
Initializing shared logging Done
Initializing OAI Logging
Initializing OAI logging Done
mme_config_parse_opt_line mme_config.config_file /usr/local/etc/oai/mme.conf
000006 00000:541475 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1016    ==== EURECOM MME vBranch: master Abrev. Hash: 6a7a8719 Date: Wed May 22 16:11:57 2019 +0200 ====
000007 00000:541637 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1018    Built with CMAKE_BUILD_TYPE ................: Debug
000008 00000:541810 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1019    Built with DISABLE_ITTI_DETECT_SUB_TASK_ID .: 1
000009 00000:541938 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1020    Built with ITTI_TASK_STACK_SIZE ............: 2097152
000010 00000:542074 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1021    Built with ITTI_LITE .......................: 0
000011 00000:542201 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1022    Built with LOG_OAI .........................: 1
000012 00000:542329 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1023    Built with LOG_OAI_CLEAN_HARD ..............: 0
000013 00000:542456 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1024    Built with MESSAGE_CHART_GENERATOR .........: 0
000014 00000:542589 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1025    Built with PACKAGE_NAME ....................: MME
000015 00000:542716 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1026    Built with S1AP_DEBUG_LIST .................: 0
000016 00000:542843 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1027    Built with SECU_DEBUG ......................: 1
000017 00000:542968 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1028    Built with SCTP_DUMP_LIST ..................: 0
000018 00000:543094 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1029    Built with TRACE_HASHTABLE .................: 0
000019 00000:543221 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1030    Built with TRACE_3GPP_SPEC .................: 1
000020 00000:543349 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1032    Configuration:
000021 00000:543476 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1033    - File .................................: /usr/local/etc/oai/mme.conf
000022 00000:543604 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1034    - Realm ................................: ng4T.com
000023 00000:543753 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1035    - Run mode .............................: BASIC
000024 00000:543881 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1036    - Max eNBs .............................: 8
000025 00000:544009 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1037    - Max UEs ..............................: 128
000026 00000:544136 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1038    - IMS voice over PS session in S1 ......: false
000027 00000:544274 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1039    - Emergency bearer services in S1 mode .: false
000028 00000:544400 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1040    - Location services via epc ............: false
000029 00000:544529 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1041    - Extended service request .............: false
000030 00000:544657 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1042    - Unauth IMSI support ..................: false
000031 00000:544783 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1043    - Relative capa ........................: 10
000032 00000:544918 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1044    - Statistics timer .....................: 10 (seconds)

000033 00000:545069 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1045    - S1-MME:
000034 00000:545188 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1046        port number ......: 36412
000035 00000:545299 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1047    - IP:
000036 00000:545418 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1048        s1-MME iface .....: ens3:m1c
000037 00000:545530 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1049        s1-MME ip ........: 192.168.247.102
000038 00000:545650 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1050        s11 MME iface ....: ens3:m11
000039 00000:545761 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1051        s11 MME port .....: 2123
000040 00000:545882 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1052        s11 MME ip .......: 172.16.1.102
000041 00000:545994 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1053        s10 MME iface ....: ens3:m10
000042 00000:546122 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1054        s10 MME port .....: 2123
000043 00000:546235 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1055        s10 MME ip .......: 192.168.10.110
000044 00000:546354 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1056    - ITTI:
000045 00000:546467 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1057        queue size .......: 2000000 (bytes)
000046 00000:546587 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1058        log file .........: (null)
000047 00000:546699 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1059    - SCTP:
000048 00000:546819 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1060        in streams .......: 8
000049 00000:546929 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1061        out streams ......: 8
000050 00000:547050 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1062    - GUMMEIs (PLMN|MMEGI|MMEC):
000051 00000:547161 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1065                208.93 |32768|3 
000052 00000:547281 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1068    - TAIs : (mcc.mnc:tac)
000053 00000:547391 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1071    - TAI list type one PLMN consecutive TACs
000054 00000:547509 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1083                208. 93:600
000055 00000:547620 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1083                208. 93:601
000056 00000:547800 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1083                208. 93:602
000057 00000:547911 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1089    - NAS:
000058 00000:548032 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1094        Prefered Integrity Algorithms .: EIA2 EIA1 EIA0 EIA0 (decreasing priority)
000059 00000:548143 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1099        Prefered Integrity Algorithms .: EEA0 EEA1 EEA2 EEA0 (decreasing priority)
000060 00000:548271 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1100        T3346 ....: 5 min
000061 00000:548380 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1101        T3402 ....: 12 min

Broadcast message from systemd-journald@test-install-openair-cn (Thu 2019-05-23 14:11:03 UTC):

mme[18197]: 000000 00000:541142 7F65369E42C0 DEBUG CONFIG nair-cn/src/mme_app/mme_config.c:1240    Config file is (null)


Broadcast message from systemd-journald@test-install-openair-cn (Thu 2019-05-23 14:11:03 UTC):

mme[18197]: 000001 00000:541427 7F65369E42C0 INFO  MME-AP nair-cn/src/mme_app/mme_config.c:0791    Parsing configuration file found S1-MME: 192.168.247.102/24 on ens3:m1c

000062 00000:550537 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1102        T3412 ....: 54 min
000063 00000:550545 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1103        T3422 ....: 6 sec
000064 00000:550549 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1104        T3450 ....: 6 sec
000065 00000:550553 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1105        T3460 ....: 6 sec
000066 00000:550556 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1106        T3470 ....: 6 sec
000067 00000:550559 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1107        T3485 ....: 8 sec
000068 00000:550562 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1108        T3486 ....: 8 sec
000069 00000:550566 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1109        T3489 ....: 4 sec
000070 00000:550569 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1110        T3470 ....: 6 sec
000071 00000:550572 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1111        T3495 ....: 8 sec
000072 00000:550579 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1112        NAS non standart features .:
000073 00000:550583 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1113          Force TAU ...................: true
000074 00000:550586 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1114          Force reject SR .............: true
000075 00000:550590 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1115          Disable Esm information .....: false
000076 00000:550593 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1117    - S6A:
000077 00000:550596 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1118        conf file ........: /usr/local/etc/oai/freeDiameter/mme_fd.conf
000078 00000:550600 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1119    - Logging:
000079 00000:550604 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1120        Output ..............: CONSOLE
000080 00000:550607 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1121        Output thread safe ..: false
000081 00000:550611 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1122        Output with color ...: true
000082 00000:550614 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1123        UDP log level........: TRACE
000083 00000:550618 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1124        GTPV2-C log level....: TRACE
000084 00000:550621 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1125        SCTP log level.......: TRACE
000085 00000:550624 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1126        S1AP log level.......: TRACE
000086 00000:550628 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1127        ASN1 Verbosity level : 0
000087 00000:550631 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1128        NAS log level........: TRACE
000088 00000:550635 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1129        MME_APP log level....: TRACE
000089 00000:550638 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1130        S10 log level........: INVALID_LOG_LEVEL
000090 00000:550642 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1131        S11 log level........: TRACE
000091 00000:550645 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1132        S6a log level........: TRACE
000092 00000:550649 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1133        UTIL log level.......: TRACE
000093 00000:550652 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1134        MSC log level........: ERROR (MeSsage Chart)
000094 00000:550656 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1135        ITTI log level.......: ERROR (InTer-Task Interface)
000095 00000:550659 7F65369E42C0 INFO  CONFIG nair-cn/src/mme_app/mme_config.c:1136        XML log level........: INVALID_LOG_LEVEL (XML dump/load of messages)

Broadcast message from systemd-journald@test-install-openair-cn (Thu 2019-05-23 14:11:03 UTC):

mme[18197]: 000002 00000:541441 7F65369E42C0 INFO  MME-AP nair-cn/src/mme_app/mme_config.c:0807    Parsing configuration file found S11: 172.16.1.102/24 on ens3:m11

 Creating queue of message of size 128
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 256
 Creating queue of message of size 1024
 Creating queue of message of size 1024
 Creating queue of message of size 256

Broadcast message from systemd-journald@test-install-openair-cn (Thu 2019-05-23 14:11:03 UTC):

mme[18197]: 000003 00000:541448 7F65369E42C0 INFO  MME-AP nair-cn/src/mme_app/mme_config.c:0822    Parsing configuration file found S10: 192.168.10.110/24 on ens3:m10


Broadcast message from systemd-journald@test-install-openair-cn (Thu 2019-05-23 14:11:03 UTC):

mme[18197]: 000004 00000:541459 7F65369E42C0 INFO  MME-AP nair-cn/src/mme_app/mme_config.c:0962    Parsing configuration file found S-GW S11: 172.16.1.104


Broadcast message from systemd-journald@test-install-openair-cn (Thu 2019-05-23 14:11:03 UTC):

mme[18197]: 000005 00000:541465 7F65369E42C0 INFO  MME-AP nair-cn/src/mme_app/mme_config.c:0962    Parsing configuration file found S-GW S11: 172.16.1.104

000096 00000:569138 7F65369E42C0 DEBUG NAS    ir-cn/src/nas/emm/nas_emm_task.c:0171    Initializing NAS EMM task interface
000097 00000:569177 7F65369E42C0 TRACE NAS-EM penair-cn/src/nas/emm/emm_main.c:0092    Entering emm_main_initialize()
000098 00000:569193 7F65369E42C0 TRACE NAS    air-cn/src/nas/api/mme/mme_api.c:0120      Entering mme_api_get_emm_config()
000099 00000:569204 7F65369E42C0 TRACE NAS    air-cn/src/nas/api/mme/mme_api.c:0250      Leaving mme_api_get_emm_config() (rc=0)
000100 00000:569225 7F65369E42C0 TRACE NAS-EM penair-cn/src/nas/emm/emm_main.c:0109    Leaving emm_main_initialize()
...
...
...
000287 00000:642737 7F6525FFB700 ERROR S6A    tu/openair-cn/src/s6a/s6a_task.c:0089    ...... 00000:642735 7F6525FFB700 LEVEL S6A --no source file-- No TLS protection negotiated with peer 'hss.ng4T.com'.
000288 00000:642765 7F6525FFB700 ERROR S6A    tu/openair-cn/src/s6a/s6a_task.c:0089    ...... 00000:642763 7F6525FFB700 LEVEL S6A --no source file-- 'STATE_WAITCEA'	-> 'STATE_OPEN'	'hss.ng4T.com'
000289 00001:636354 7F64DDFFB700 DEBUG S6A    tu/openair-cn/src/s6a/s6a_peer.c:0116    Diameter identity of MME: mme.ng4T.com with length: 12
000290 00001:636400 7F64DDFFB700 DEBUG S6A    tu/openair-cn/src/s6a/s6a_peer.c:0151    S6a peer connection attempt 1 / 8
000291 00001:636417 7F64DDFFB700 DEBUG S6A    tu/openair-cn/src/s6a/s6a_peer.c:0164    Peer hss.ng4T.com is now connected...
000292 00001:636597 7F652E9EF700 DEBUG SCTP   rc/sctp/sctp_primitives_server.c:0674    Received SCTP_INIT_MSG
000293 00001:636621 7F652E9EF700 DEBUG SCTP   rc/sctp/sctp_primitives_server.c:0320    Creating new listen socket on port 36412 with
000294 00001:636626 7F652E9EF700 DEBUG SCTP   rc/sctp/sctp_primitives_server.c:0325    ipv4 addresses:
000295 00001:636632 7F652E9EF700 DEBUG SCTP   rc/sctp/sctp_primitives_server.c:0334    	- 192.168.247.102
000296 00010:580341 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0045    ======================================= STATISTICS ============================================

000297 00010:580364 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0046                   |   Current Status| Added since last display|  Removed since last display |
000298 00010:580370 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0048    Connected eNBs |          0      |              0              |             0               |
000299 00010:580375 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0050    Attached UEs   |          0      |              0              |             0               |
000300 00010:580379 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0052    Connected UEs  |          0      |              0              |             0               |
000301 00010:580384 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0054    Default Bearers|          0      |              0              |             0               |
000302 00010:580388 7F6527FFF700 DEBUG MME-AP src/mme_app/mme_app_statistics.c:0056    S1-U Bearers   |          0      |              0              |             0               |
...
...

```


**Run SPGW-C**

Run SPGW-C prior to SPGW-U.
```
ubuntu@test-install-openair-cn:~/openair-cn-cups/build/scripts$ sudo spgwc -c $PREFIX/spgw_c.conf

```


**Run SPGW-U**

**Reminder: Run SPGW-C prior to SPGW-U.**

You will have to do the following only once, each time you power on your host if you want to set a specific gateway for user traffic:

```
echo '200 lte' | sudo tee --append /etc/iproute2/rt_tables
# Here the gateway is at 192.168.78.245
sudo ip r add default via 192.168.78.245 dev ens10 table lte
# you will have to repeat the following line for each PDN network set in your SPGW-U config file
sudo ip rule add from 12.0.0.0/8 table lte
```


```
ubuntu@test-install-openair-cn:~/openair-cn-cups/build/scripts$ sudo spgwu -c $PREFIX/spgw_u.conf
```

## Troubleshooting

### Install HSS 
#### Error when execute ./build_cassandra --check-installed-software --force
**~~TODO: Check if JAVA installation really needs. The cassandra install open _java-8-openjdk-amd64_ package.~~**   
**CHECKED!! JAVA is intalled with cassandra package. DO NOT NEED TO INTALL IT!!**

Fix: https://github.com/OPENAIRINTERFACE/openair-cn/pull/93 or you can see other solution in the sections below.


#### How to Install Oracle JAVA JDK8 on Ubuntu
https://github.com/navarrothiago/masters/blob/master/scripts/download_install_jdk.sh

#### Check Oracle JAVA JDK8 on Ubuntu
```
java -version
```
The output of the command above should be: 
```
java version "1.8.0_241"
Java(TM) SE Runtime Environment (build 1.8.0_241-b07)
Java HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)
```

#### Cassandra - no valid OpenPGP data found
Updates https://github.com/OPENAIRINTERFACE/openair-cn/blob/master/build/tools/build_helper.cassandra
```
Replace
curl https://www.apache.org/dist/cassandra/KEYS | $SUDO  apt-key add - 
with
curl https://downloads.apache.org/cassandra/KEYS | $SUDO apt-key add -  
```

#### Cannot achieve consistency level LOCAL_ONE
Other problem is shown below
```
cd ~/openair-cn/scripts/
Cassandra_Server_IP='127.0.0.1'
cqlsh --file ../src/hss_rel14/db/oai_db.cql $Cassandra_Server_IP
openair-cn@openair-cn-vm:~/openair-cn/scripts$ ./data_provisioning_users --apn default.ng4T.com --apn2 internet --key fec86ba6eb707ed08905757b1bb44b8f --imsi-first 208931234561000 --msisdn-first 001011234561000 --mme-identity mme.ng4T.com --no-of-users 20 --realm ng4T.com --truncate True  --verbose True --cassandra-cluster $Cassandra_Server_IP

Traceback (most recent call last):
  File "./data_provisioning_users", line 178, in <module>
    main()
  File "./data_provisioning_users", line 155, in main
    '{"Subscription-Data":{"Access-Restriction-Data":41,"Subscriber-Status":0,"Network-Access-Mode":2,"Regional-Subscription-Zone-Code":["0x0123","0x4567","0x89AB","0xCDEF","0x1234","0x5678","0x9ABC","0xDEF0","0x2345","0x6789"],"MSISDN":"0x'+str(msisdn)+'","AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"APN-Configuration-Profile":{"Context-Identifier":0,"All-APN-Configurations-Included-Indicator":0,"APN-Configuration":{"Context-Identifier":0,"PDN-Type":0,"Service-Selection":"'+args.apn+'","EPS-Subscribed-QoS-Profile":{"QoS-Class-Identifier":9,"Allocation-Retention-Priority":{"Priority-Level":15,"Pre-emption-Capability":0,"Pre-emption-Vulnerability":0}},"AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"PDN-GW-Allocation-Type":0,"MIP6-Agent-Info":{"MIP-Home-Agent-Address":["172.26.17.183"]}},"APN-Configuration":{"Context-Identifier":0,"PDN-Type":0,"Service-Selection":"'+args.apn2+'","EPS-Subscribed-QoS-Profile":{"QoS-Class-Identifier":9,"Allocation-Retention-Priority":{"Priority-Level":13,"Pre-emption-Capability":1,"Pre-emption-Vulnerability":0}},"AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"PDN-GW-Allocation-Type":0,"MIP6-Agent-Info":{"MIP-Home-Agent-Address":["172.26.17.183"]}}},"Subscribed-Periodic-RAU-TAU-Timer":0}}'
  File "/usr/lib/python3/dist-packages/cassandra/cluster.py", line 1998, in execute
    return self.execute_async(query, parameters, trace, custom_payload, timeout, execution_profile, paging_state).result()
  File "/usr/lib/python3/dist-packages/cassandra/cluster.py", line 3781, in result
    raise self._final_exception
cassandra.cluster.NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 dc1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level LOCAL_ONE" info={\'consistency\': \'LOCAL_ONE\', \'required_replicas\': 1, \'alive_replicas\': 0}',)})
openair-cn@openair-cn-vm:~/openair-cn/scripts$ 

```
 After change cassandra configuration (**/etc/cassandra/cassandra.yaml**) according to commands below, it works!
```
endpoint_snitch: SimpleSnitch
#endpoint_snitch: GossipingPropertyFileSnitch
```

If you already have this configuration, see the database file in **../src/hss_rel14/db/oai_db.cql** and check if the name the datacenter is _datacenter1_. The first line should be:
```
CREATE KEYSPACE IF NOT EXISTS vhss WITH replication = {'class': 'NetworkTopologyStrategy', 'datacenter1': '1'};
```
This name is the same when you run _nodetool status_
```
epc@epc-vm:~/openair-cn/scripts$ nodetool status
Datacenter: datacenter1
=======================
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack
UN  127.0.0.1  339.73 KB  256     100,0%            b87fc37f-b416-43f9-9a81-06476ec2a3b7  rack1

```


#### Failed opening file logs/hss.log for writing: No such file or directory 
Open this file
```
sudo nano /usr/local/etc/oai/hss_rel14.json
```
you will see some config where you can find logs/hss.log

actually you have to change these 4 value to
```
logname: "/var/log/hss.log"
statlogname: "/var/log/hss_stat.log"
auditlogname: "/var/log/hss_audit.log"
ossfile: "~/openair-cn/etc/oss.json"
```
then use <i>sudo touch</i> to create these files
```
sudo touch /var/log/hss.log
sudo touch /var/log/hss_stat.log
sudo touch /var/log/hss_audit.log
```
for logname, statlogname, and auditlogname you can change it to whatever file you want but i like to put them together in /var/log folder.
for ossfile , the oss.json is actually in there.
hope this help

### Cassandra Startup Error - java.io.FileNotFoundException: /var/log/cassandra/system.log (Permission denied)
Run the command in order to see if there are any error
```
dmesg|grep cassandra
```
If you found the error described in the section, run 
```
sudo chown -R cassandra:cassandra /var/lib/cassandra/
```

### Cassandra Startup Error - org.apache.cassandra.exceptions.ConfigurationException: Saved cluster name xxxx != yyyyy

Run the command in order to see if there are any error
```
dmesg|grep cassandra
```
Stop Cassandra and cleanup the log files before modifying the configuration

```
sudo service cassandra stop
sudo rm -rf /var/lib/cassandra/data/system/*
sudo rm -rf /var/lib/cassandra/commitlog/*
sudo rm -rf /var/lib/cassandra/data/system_traces/*
sudo rm -rf /var/lib/cassandra/saved_caches/*
```

Check the **/etc/cassandra/cassandra.yaml** if the cluster_name is what you have programed

### Install SPGW-U
#### _CMake Error at CMake/folly-deps.cmake:213 \n No "Findfmt.cmake" foun in CMAKE_MODULE_PATH
If the error is shown when execute the command below
```
ubuntu@test-install-openair-cn:~/openair-cn-cups/build/scripts$ ./build_spgwu -I -f
```
execute the following steps
```
cd ~
git clone https://github.com/fmtlib/fmt.git && cd fmt
mkdir _build && cd _build
cmake ..
make -j$(nproc)
sudo make install
```

## Manage Sections - tmux